In [2]:
from peft import IA3Model, IA3Config, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
#from auto_gptq import AutoGPTQForCausalLM
import gc

In [3]:
gc.collect()

74

In [4]:
BASE_MODEl = '/home/dzigen/Desktop/ITMO/sem1/DLtech/dl_tech_learn/lab3/pretrained_models/ruGPT3_5_13B_8bit'
DEVICE = 'cpu'
BATCH_SIZE = 1
EPOCHS = 12
LR = 3e-4

print("BATCH_SIZE: ", BATCH_SIZE)
print("LR: ", LR)
print("EPOCHS: ", EPOCHS)
print("DEVICE: ", DEVICE)

In [5]:
CONFIG = IA3Config(
        peft_type="IA3",
        task_type="CAUSAL_LM",
        target_modules=["c_attn", "c_proj","c_fc"],
        feedforward_modules=['c_fc'])

MODEL = AutoModelForCausalLM.from_pretrained(BASE_MODEl, device_map=DEVICE, 
                                             load_in_8bit=True, use_safetensors=True)

# frozing full base model
for param in MODEL.parameters():
    param.requires_grad = False

# wraps model to ia3
IA3_MODEL = get_peft_model(MODEL, CONFIG)

for name, param in IA3_MODEL.named_parameters():
    print(name, param.requires_grad)

# checking trainable parameters
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(IA3_MODEL)

In [ ]:
# prepare dataset

In [ ]:
# loop
## train
## eval
## save best